In [9]:
%%writefile matrix_2310336.txt

9,8
7,6,9,8
4,3,7,9
8,7,9,6

2,1
8,7,6
9,3,7
6,4,9
3,7,4

8,9
6.2,8.7,9.5
7.4,3.5,9.2
2.1,6.1,4.5
7.5,3.4,6.9
4.3,7.9,6.7

9,7
8.1,6.4,3.2,5.6,9.8
7.3,6.2,6.7,3.2,7.6
1.9,8.5,6.4,3.5,6.9

2,5
234,567,876,345,123
456,789,321,543,876
234,654,987,765,234
567,432,765,987,543

5,8
84,27,43,98,28,57,23,35
47,91,37,28,46,13,37,68
25,64,23,82,93,17,39,81
32,13,26,47,81,54,62,35
82,46,74,35,51,23,79,14
72,95,18,57,33,86,28,74
43,56,98,42,47,12,63,22

8,10
47,35,81,37,25,68,42,24,65,27
14,92,78,46,23,35,49,67,26,63
83,38,64,74,36,26,47,53,28,15
95,42,19,84,25,37,69,48,23,77
33,56,79,48,92,15,63,21,43,79
84,36,98,21,32,54,67,13,54,28
46,23,81,47,29,15,74,23,35,89

10,7
48,37,59,28,75,92,61
19,28,35,87,63,47,84
52,16,37,68,42,35,78
29,84,47,59,36,85,47
18,53,24,92,74,59,82
65,74,32,84,39,58,91
46,95,42,28,84,27,22
29,36,82,94,28,67,94
47,39,68,15,36,71,66
27,42,19,73,28,46,91

Overwriting matrix_2310336.txt


In [10]:
%%writefile Matrix-Multiplication.c

#include <stdlib.h>
#include <stdio.h>
#include <pthread.h>
#include <string.h>

int bufferSize = 256;


struct MatrixSize {
    int m;
    int n;
};


struct Matrix {
    struct MatrixSize size;
    double* values;  
};

struct Matrix** Matrices;


struct threadArguments {
    int currentThreadIndex;
    int iterateCount;
    struct Matrix* matrixInfo;
    double* allMtxAVals;
    double* allMtxBVals;
};


int emptyCharArray (char * chars, int size) {
    return chars[0] == '\n';
}


int FileMatrixCount(char* fileName) {
    FILE * fPtr;
    fPtr = fopen(fileName, "r");
    if (fPtr == NULL) {
        printf("Unable to load file '%s'\n", fileName);
        return 0;
    }

    int matrixCount = 0;
    char readBuffer[bufferSize];

    for (int i = 0; fgets(readBuffer, bufferSize, fPtr) != NULL; i++) {
        if (emptyCharArray(readBuffer, bufferSize) > 0) {
            matrixCount++;
        }
    }

    matrixCount++;

    fclose(fPtr);

    return matrixCount;
}


struct MatrixSize* charArrayIsMatrixSize (char * chars, int size) {
    struct MatrixSize* mtxSize = (struct MatrixSize*) malloc( sizeof(struct MatrixSize) );
    int splitCount = 0;

    char temp[bufferSize];
    strcpy(temp, chars);

    char* split = strtok(temp, ",");
    while (split != NULL) {
        double interpreted = atof(split);
        if ( interpreted > 0 )
        {
            if (splitCount == 0) {
                (*mtxSize).m = interpreted;
            } else if (splitCount == 1) {
                (*mtxSize).n = interpreted;
            } else {
                // More commas than expected. Is a matrix value line
                return NULL;
            }
            splitCount++;
        }

        // Move split onto next split index
        split = strtok(NULL, ",");
    }
    return mtxSize;
}


double* parseLineOfDoubles(char* lineChars, struct MatrixSize mtxSize) {
    double* lineVals = malloc (sizeof(double) * mtxSize.n);
    int lineYCount = 0;

    const char splitter[2] = ",";
    char* split = strtok(lineChars, splitter);
    while (split) {
        double interpreted = atof(split);
        if ( interpreted > 0 ) {
            lineVals[lineYCount] = interpreted;
            lineYCount += 1;
        }


        split = strtok(NULL, splitter);
    }

    return lineVals;
}


struct Matrix** loadFromFile (char * fileName) {
    if (fileName) {
 printf("File '%s' opened successfully in read mode.\n", fileName);

      int matrixCount = FileMatrixCount(fileName);

      FILE * fPtr;
      fPtr = fopen(fileName, "r");
      if (fPtr == NULL) {
          printf("Cannot load file '%s'\n", fileName);
          return NULL;
      }
      struct Matrix** matrices = malloc(sizeof(struct Matrix) * matrixCount);

      struct MatrixSize* currentMatrixSize = (struct MatrixSize*) malloc(sizeof(struct MatrixSize));
      (*currentMatrixSize).m = (*currentMatrixSize).n = 0;

      double* currentMatrixValues = NULL;
      int currentMatrixCount = 0;
      int currentMatrixCurrentLineCount = 0;

      char readBuffer[bufferSize];
      while ( fgets(readBuffer, bufferSize, fPtr) != NULL ) {
          struct MatrixSize* mtxSize = charArrayIsMatrixSize(readBuffer, bufferSize);
          if ( mtxSize != NULL && (*currentMatrixSize).m == 0 && (*currentMatrixSize).n == 0 ) {
              currentMatrixSize = mtxSize;
          } else if ( emptyCharArray(readBuffer, bufferSize) > 0 ) {

              struct Matrix* currentMatrix = malloc( sizeof(struct Matrix) );
              (*currentMatrix).values = currentMatrixValues;
              (*currentMatrix).size = *currentMatrixSize;

              matrices[currentMatrixCount] = currentMatrix;

              currentMatrixCount++;

         
              currentMatrixCurrentLineCount = 0;
              (*currentMatrixSize).m = (*currentMatrixSize).n = 0;
              currentMatrixValues = NULL;
          } else {
              if (currentMatrixValues == NULL) {
                  currentMatrixValues = malloc( sizeof(double) * (*currentMatrixSize).n);
              }

              double* values = parseLineOfDoubles(readBuffer, *currentMatrixSize);

              for (int i = 0; i < (*currentMatrixSize).n; i++) {
                  int index = (currentMatrixCurrentLineCount * (*currentMatrixSize).n) + i;
                  currentMatrixValues[index] = values[i];
              }
              currentMatrixCurrentLineCount++;
          }
      }

      struct Matrix* currentMatrix = malloc( sizeof(struct Matrix) );
      (*currentMatrix).values = currentMatrixValues;
      (*currentMatrix).size = *currentMatrixSize;

      matrices[currentMatrixCount] = currentMatrix;

      currentMatrixCount++;

      fclose(fPtr);

      return matrices;
    } else {
        printf("Cannot find file '%s'\n", fileName);
        return NULL;
    }
}

int isMultiply(struct Matrix a, struct Matrix b);

int isMultiply(struct Matrix a, struct Matrix b) {
    if (a.size.n == b.size.m) {
        return 1;
    }
    return 0;
}

void* calculate(void* param);

void* calculate (void* param) {

    struct threadArguments* args = (struct threadArguments*)param;

    struct MatrixSize matrixSizeOutput = (*args).matrixInfo->size;
    int index = (*args).currentThreadIndex;

    int column = index %  matrixSizeOutput.m;
    int row = (index - column) / matrixSizeOutput.n;

    
    double result = 0;
    for (int i = 0; i < (*args).iterateCount; i++) {
     
        int mtxAThisIndex = (row * (matrixSizeOutput.m - 1)) + i;
        int mtxBThisIndex = (i * matrixSizeOutput.n) + column;

   
        double finalA = (*args).allMtxAVals[ mtxAThisIndex ];
        double finalB = (*args).allMtxBVals[ mtxBThisIndex ];
        result += finalA * finalB;
    }

    (*args).matrixInfo->values[(*args).currentThreadIndex] = result;

    return NULL;
}

void printMatrix(double* matrix, int countM, int countN);

void printMatrix(double* matrix, int countM, int countN) {
    for (int i = 0; i < countM; i++) {
        for(int j = 0; j < countN; j++) {
            printf("%.2f", *(matrix + (i * countN) + j));

          
            if ( j < countN - 1 ) {
                printf("    ");
            }
        }
        printf("\n");
    }
}

int saveToFile (char* inputFile, struct Matrix** matrices, int matricesCount) {

    FILE* writeFile = fopen(inputFile, "w");
    if (writeFile == NULL)
    {
        printf("Unable to open file '%s'.\n", inputFile);
        return 0;
    }

    fprintf(writeFile, "matrixresults2314773\n\n");

    for( int i = 0; i < matricesCount; i++) {
        struct Matrix matrix = *(matrices[i]);

        fprintf(writeFile,"Matrix[%d] X Matrix[%d]\n----------------------\n\n", i, i + 1);

        for (int j = 0; j < matrix.size.m; j++) {
            int rowIndex = (j * matrix.size.n);
            for( int k = 0; k < matrix.size.n; k++ ) {
                fprintf(writeFile, "%.2f", matrix.values[rowIndex + k]);
                if (k < matrix.size.n - 1) {
                    fprintf(writeFile, "   ");
                }
            }
            fprintf(writeFile, "\n");
        }
        fprintf(writeFile, "\n");
    }
    fclose(writeFile);
    return 1;
}

int main () {

    char* inputFile = "./matrix_2310336.txt";

    char* outputFile = "./matrixresults2310336.txt";

    int matrixCount = FileMatrixCount(inputFile);
    Matrices = loadFromFile(inputFile);

    printf("%d Matrices found\n\n", matrixCount);

    struct Matrix** allFinalMatrices = malloc(sizeof(struct Matrix) * matrixCount);
    int allMatricesCount = 0;

    for (int i = 0; i < matrixCount; i += 2) {
        struct Matrix matrixA = *(Matrices[i]);
        struct Matrix matrixB = *(Matrices[i + 1]);

        struct Matrix* matrixResult = malloc(sizeof(struct Matrix));
        (*matrixResult).size.m = matrixA.size.m;
        (*matrixResult).size.n = matrixB.size.n;

        (*matrixResult).values = malloc(sizeof(double) * (*matrixResult).size.m * (*matrixResult).size.n);

        int canMultiply = isMultiply(matrixA, matrixB);
        if (canMultiply == 1)
        {
            int threadCount = (*matrixResult).size.m * (*matrixResult).size.n;

            pthread_t* threadIds = malloc( sizeof(pthread_t) * threadCount);

            for (int i = 0; i < threadCount; i++) {
                struct threadArguments *args = malloc(sizeof(struct threadArguments));
                if (args != NULL)
                {
                    (*args).currentThreadIndex = i;
                    (*args).iterateCount = matrixA.size.n;

                    (*args).matrixInfo = matrixResult;
                    (*args).allMtxAVals = matrixA.values;
                    (*args).allMtxBVals = matrixB.values;

                    pthread_create( &threadIds[i], NULL, calculate, args);
                }
            }

            printf("\n*********************************************************************************************\n");
            printf("%dX%d X %dX%d\n",  matrixA.size.m, matrixA.size.n, matrixB.size.m, matrixB.size.n);

         
            for (int i = 0; i < threadCount; i++) {
                pthread_join( threadIds[i], NULL );
            }

            printf("Matrix[%d] X Matrix[%d]\n", i, i + 1);
            printf("-----------------------\n");

            printMatrix((*matrixResult).values, (*matrixResult).size.m, (*matrixResult).size.n);

            printf("\n%d threads created\n", threadCount);

            allFinalMatrices[allMatricesCount] = matrixResult;
            allMatricesCount++;
        }
        else
        {
            printf("\n*********************************************************************************************\n");
            printf("Cannot multiply Matrices[%d] and Matrices[%d] because of %dx%d and %dx%d matrices are not same\n", i, i + 1, matrixA.size.m, matrixA.size.n, matrixB.size.m, matrixB.size.n);
        }
    }

    if (saveToFile(outputFile, allFinalMatrices, allMatricesCount)) {
        printf("Saved '%d' matrices on '%s'\n", allMatricesCount, outputFile);
    } else {
        printf("Can't save file '%s'\n", outputFile);
    }
    return 0;
}


Overwriting Matrix-Multiplication.c


In [11]:
!gcc Matrix-Multiplication.c -pthread -o matrix
!./matrix


File './matrix_2310336.txt' opened successfully in read mode.
9 Matrices found


*********************************************************************************************
Cannot multiply Matrices[0] and Matrices[1] because of 9x8 and 2x1 matrices are not same

*********************************************************************************************
8X9 X 9X7
Matrix[2] X Matrix[3]
-----------------------
131.78    174.37    138.93    95.81    192.43    0.00    0.00
120.69    35.76    86.35    58.56    52.50    68.54    78.20
58.88    78.08    21.35    42.09    0.00    0.00    3.99
17.85    13.44    7.35    1614.60    3912.30    6044.40    2380.50
848.70    3146.40    5444.10    3010.50    1084.11    2502.64    3823.91
1524.62    614.66    1960.80    3392.70    1418.23    87.94    138.70
100.81    92.33    87.19    0.00    0.00    57.15    0.00
0.00    0.00    0.00    0.00    0.00    0.00    0.00

56 threads created

***************************************************************